# Actions

In [13]:
import os
import re
import csv
from urllib import urlopen
import sys
import csv
from collections import Counter
from __future__ import division

csv.field_size_limit(sys.maxsize)

9223372036854775807

In [14]:
# load data
recs = []
dir = '.'
with open("../Data/upr-orig.csv",'rU') as csvfile:
            reader = csv.DictReader(csvfile)
            for row in reader:
                recs.append(row)
            
print len(recs) # 42013 
recs[0]

39329


{'Decision': 'support',
 'From': 'Brazil',
 'From_COW': 'BRA',
 'Institution': "['sp-pr']",
 'Institutions': '[]',
 'Report_Year': '2008',
 'Session': '1',
 'Session_Year': '2008',
 'Text': '69.16 That Algeria consider facilitating visits by human rights mandate holders (Brazil). );',
 'To': 'Algeria',
 'To_COW': 'ALG',
 'UID': 'ALG-BRA-1-1'}

In [16]:
for i in recs:
   del  i['Institutions']

In [17]:
recs[1].keys()

['From_COW',
 'Session_Year',
 'To_COW',
 'Text',
 'Decision',
 'UID',
 'To',
 'Session',
 'From',
 'Institution',
 'Report_Year']

In [18]:
# strip extra whitespace on everything
for i in recs:
    i['Text']=' '.join(i['Text'].split())

In [19]:
# remove stop words function
stopwords = ['of','on','all','the','a','for','to','and','to','against','from','in','or']

def noStop(in_text):
    '''
    this function inputs a text, lowers it, and removes stop words
    '''
    text = in_text.lower().split()
    for word in stopwords:
        text = [x for x in text if x != word]
    return(' '.join(text))

## Make Action Dictionary

In [20]:
# assign action
keywords = {}
keywords['ratify'] = ['ratif']
keywords['accede'] = ['acced']
keywords['sign'] = ['sign ']
keywords['implement'] = ['implement']
keywords['establish'] = ['establish']
keywords['continue'] = ['continue']
keywords['take-measures'] = ['take measures']
keywords['harmonize'] = ['harmoniz']
keywords['report'] = ['report','reporting','reports']
keywords['party'] = ['party']
keywords['withdraw'] = ['withdraw']
keywords['cooperate'] = ['cooperat']
keywords['share'] = ['share', 'sharing']

print (keywords.keys())


['accede', 'ratify', 'take-measures', 'share', 'sign', 'report', 'continue', 'withdraw', 'harmonize', 'party', 'implement', 'cooperate', 'establish']


In [21]:
for i in recs:
    i['Action_Verb'] = []
    for key in keywords.keys():
        for word in keywords[key]:
            if key in i['Action_Verb']: # skip if it's already in there
                continue
            if word in noStop(i['Text']):
                i['Action_Verb'].append(key)

In [22]:
# special rules 

# international assistance
for i in recs:
    if (
        ('assistance ' in i['Text'] or 'support ' in i['Text']) and
        'international' in i['Text']
        ):
        i['Action_Verb'].append('assistance')
        

# take ____ measures:
for i in recs:
    if 'take-measures' not in i['Action_Verb']:
        text = i['Text'].lower().replace(',','').split()
        if 'take' in text:
            takeIndex = text.index('take')
            if 'measures' in text[takeIndex:takeIndex+10]:
                i['Action_Verb'].append('take-measures')
        elif 'taking' in text:
            takeIndex = text.index('taking')
            if 'measures' in text[takeIndex:takeIndex+10]:
                i['Action_Verb'].append('take-measures')

In [23]:
# count the actions
actions = []
for rec in recs:
    for i in rec['Action_Verb']:
        actions.append(i)
Counter(actions)

Counter({'continue': 7006, 'ratify': 4642, 'implement': 4161, 'take-measures': 2554, 'establish': 2325, 'cooperate': 1310, 'accede': 817, 'report': 801, 'sign': 584, 'assistance': 572, 'party': 505, 'share': 269, 'withdraw': 244, 'harmonize': 225})

## WRITE CSV

In [24]:
#writing column headings
keys = recs[0].keys()
keys

['From_COW',
 'Action_Verb',
 'Session_Year',
 'To_COW',
 'Text',
 'Decision',
 'UID',
 'To',
 'Session',
 'From',
 'Institution',
 'Report_Year']

In [26]:
#writing the rest
with open('../Data/upr-orig.csv', 'wb') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(recs)